Modules required to run code

In [92]:
import json
import requests
from enum import Enum
import pymongo
import sys

Some definitions to keep the code cleaner

In [93]:
class KivaApi(Enum):
    BASE_URL = "https://api.kivaws.org"
    SEARCH = BASE_URL + "/v1/loans/search.json?"
    GET_LOAN = BASE_URL + "/v1/loans/"
    
class Status(Enum):
    FUNDRAISING = "fundraising"
    FUNDED = "funded"
    EXPIRED = "expired"
    NA = "na"

Simple search function to find loans

In [94]:
def search(status = Status.FUNDED):

    req = KivaApi.SEARCH.value 
    if status != Status.NA:
        req += "status=" + status.value
    
    #sample request
    #response = requests.get("http://api.kivaws.org/v1/loans/search.json?status=funded")
    response = requests.get(req)
    if response.status_code != 200:
        raise ValueError('Kiva API search failed: status_code =' + str(response.status_code))
    
    loans = json.loads(response.content).get('loans')
    
    return loans
    

To test search 

In [144]:
print(search())

[{'id': 1642943, 'name': 'Esther', 'description': {'languages': ['en']}, 'status': 'funded', 'funded_amount': 100, 'image': {'id': 2994487, 'template_id': 1}, 'activity': 'Farming', 'sector': 'Agriculture', 'themes': ['Underfunded Areas', 'Rural Exclusion', 'Vulnerable Groups', 'Innovative Loans', 'Conflict Zones'], 'use': 'to purchase more land for growing maize in order to earn higher profits next harvest and increase long-term prosperity.', 'location': {'country_code': 'NG', 'country': 'Nigeria', 'town': 'Kaduna', 'geo': {'level': 'town', 'pairs': '10.510464 7.416505', 'type': 'point'}}, 'partner_id': 288, 'posted_date': '2018-11-13T04:00:10Z', 'planned_expiration_date': '2018-12-13T04:00:10Z', 'loan_amount': 100, 'borrower_count': 1, 'lender_count': 4, 'bonus_credit_eligibility': True, 'tags': [{'name': '#Vegan'}]}, {'id': 1643228, 'name': 'Teresita', 'description': {'languages': ['en']}, 'status': 'funded', 'funded_amount': 500, 'image': {'id': 2994835, 'template_id': 1}, 'activit

Get details on one loan using id

In [150]:
def getLoanDetail(loan_id):
    req = KivaApi.GET_LOAN.value + str(loan_id) +".json"
    
    #print(req)
    
    response = requests.get(req)
    
    if response.status_code != 200:
        raise ValueError('Kiva API request failed: status_code =' + str(response.status_code))
    
    return json.loads(response.content)
    

To test getLoanDetail

In [149]:
print(getLoanDetail(1643228))

https://api.kivaws.org/v1/loans/1643228.json
{'loans': [{'id': 1643228, 'name': 'Teresita', 'description': {'languages': ['en'], 'texts': {'en': 'Teresita is 64 years old and married with one child in school, She works very hard to provide for her family.<br /><br />Teresita runs a general store in the Philippines and requested a loan of 25,000 Philippine pesos (PHP) through NWTF to buy items to sell such as beverages, canned goods, junk food, and other groceries. She has been in this business for 13 years and sells a variety of items. <br /><br />In the future, Teresita would like to save enough to provide a secure future for her family.'}}, 'status': 'funded', 'funded_amount': 500, 'image': {'id': 2994835, 'template_id': 1}, 'activity': 'General Store', 'sector': 'Retail', 'use': 'to buy items to sell such as beverages, canned goods, junk food, and other groceries.', 'location': {'country_code': 'PH', 'country': 'Philippines', 'town': 'Tacloban City, Leyte', 'geo': {'level': 'town', 

Get a list of lenders for a loan (assumes list will not be longer than one page)

In [96]:
def getLendersForLoan(loan_id):
    req = KivaApi.GET_LOAN.value + str(loan_id) +"/lenders.json"
    
    #print(req)
    
    response = requests.get(req)
    
    if response.status_code != 200:
        raise ValueError('Kiva API request failed: status_code =' + str(response.status_code))

    return json.loads(response.content).get('lenders')

Test of getLenderForLoan

In [151]:
print(getLendersForLoan(1643228))

[{'lender_id': 'mark2704', 'name': 'Mark', 'image': {'id': 1691804, 'template_id': 1}, 'whereabouts': 'Georgia', 'country_code': 'US', 'uid': 'mark2704'}, {'lender_id': 'aaron63004970', 'name': 'Aaron', 'image': {'id': 726677, 'template_id': 1}, 'whereabouts': '', 'uid': 'aaron63004970'}, {'lender_id': 'compadres9375', 'name': 'Compadres Foundation', 'image': {'id': 1002514, 'template_id': 1}, 'whereabouts': 'Temecula CA', 'country_code': 'US', 'uid': 'compadres9375'}, {'lender_id': 'JY1024', 'name': 'Johnny', 'image': {'id': 726677, 'template_id': 1}, 'whereabouts': 'Cantabrigia', 'country_code': 'US', 'uid': 'jy1024'}, {'lender_id': 'michael43411218', 'name': 'Michael', 'image': {'id': 1384783, 'template_id': 1}, 'whereabouts': 'Dallas Texas', 'country_code': 'US', 'uid': 'michael43411218'}, {'name': 'Anonymous', 'image': {'id': 726677, 'template_id': 1}}, {'lender_id': 'markus4455', 'name': 'Nullinger', 'image': {'id': 619142, 'template_id': 1}, 'whereabouts': 'Babenhausen Bavaria',

Records payments made to lenders for a specific loan (in production this would normally link to the actual payment record and round to the nearest penny).

In [140]:
def recordPayment(lenders, amount_paid):
    
    if lenders:
        amt = amount_paid/len(lenders)
    
        to_insert = []
        
        for l in lenders:
            if "lender_id" in l:
                to_insert.append({"lender_id": l["lender_id"], "amount": amt})
        
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        db = client["kivapayments"]
        db.payments.insert_many(to_insert)    

Test of recordPayment

In [141]:
d = getLendersForLoan(1643228)
recordPayment(d, 1000)

And just for fun here is a single graphql query to return list of lenders using the id of the loan. Note that the lender id uses the alias "lender_id" to be compatable with the recordPayment function above.

In [183]:
def getLendersGraphQL(loan_id):
    query = """
{
  lend {
    loan(id: """ + str(loan_id) + """)
     {
        lenders (limit: 20) {        
          values {lender_id: id}
        }
    }
  }
}
"""
    #print(query)
    url = KivaApi.BASE_URL.value + "/graphql"
    
    response = requests.post(url, json={'query':query})
    
    if response.status_code != 200:
        raise ValueError('Kiva API request failed: status_code =' + str(response.status_code))

    return response.json()["data"]["lend"]["loan"]["lenders"]["values"]

In [186]:
d = getLendersGraphQL(1643228)
recordPayment(d, 200)